# Play Store App Reviews - Doc

In [ ]:
from google_play_scraper import reviews, Sort
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

#### Setando as variávies.

In [ ]:
APP_ID = "com.netflix.mediaclient" # O App ID que queremos analisar. (Você encontra isso na URL do App no Google Play Store)
LANGUAGE = "pt-br" # Língua.
COUNTRY = "br" # País.

TOTAL_ROWS_OF_DATA = 10000 # Aqui você coloca quantas avaliações você quer pegar, lembre-se que é a partir da avaliação mais recente.

#### Rodando o request e pegando os dados.

In [ ]:
results, _ = reviews(
    APP_ID,
    lang=LANGUAGE,
    country=COUNTRY,
    sort=Sort.NEWEST,
    count=TOTAL_ROWS_OF_DATA,
    filter_score_with=None,
)

In [ ]:
data = []

for result in results:
    review_id = result["reviewId"]
    content = result["content"]
    score = result["score"]
    thumbs_up_count = result["thumbsUpCount"]
    review_created_version = result["reviewCreatedVersion"]
    at = result["at"]
    reply_content = result["replyContent"]
    replied_at = result["repliedAt"]
    app_version = result["appVersion"]

    data.append(
        {
            "Review ID": review_id,
            "Content": content,
            "Score": score,
            "Thumbs Up Count": thumbs_up_count,
            "Review Created Version": review_created_version,
            "At": at,
            "Reply Content": reply_content,
            "Replied At": replied_at,
            "App Version": app_version,
        }
    )

df = pd.DataFrame(data)

#### Exibindo as cinco primeiras linhas dos dados.

In [ ]:
df.head()

### Gráfico de distribuição das pontuações.

* Qual é a quantidade de avaliações para 1 estrela? Devemos nos preocupar? 🤔

In [ ]:
score_counts = df['Score'].value_counts()
score_counts = score_counts.sort_index()

fig = px.bar(x=score_counts.index, y=score_counts.values, title='Distribuição das pontuações.')
fig.update_layout(xaxis_title='Pontuação', yaxis_title='Contagem')

fig.show()

### Analisando por versão do aplicativo.
* Dos nossos dados, de qual versão tem mais avaliações?

In [ ]:
value_counts = df['App Version'].value_counts()

value_percentages = value_counts / len(df) * 100

filtered_values = value_counts[value_percentages >= 5]
filtered_value_percentages = value_percentages[value_percentages >= 5]

fig = px.pie(values=filtered_values, names=filtered_values.index, title="Avaliações por versão do aplicativo. (Excluído < 5%)")
fig.show()

In [ ]:
total_reviews = len(df)
app_version_counts = df['App Version'].value_counts()
threshold = total_reviews * 0.05 # Eliminando os 5% (a gordurinha :p)

filtered_app_versions = app_version_counts[app_version_counts >= threshold]
filtered_df = df[df['App Version'].isin(filtered_app_versions.index)]

score_counts = filtered_df.groupby(['App Version', 'Score']).size().reset_index(name='Count')
px.bar(score_counts, x='App Version', y='Count', color='Score', title='Contagem de Avaliações por Versão do Aplicativo e Pontuação. (Excluído 5%)',
            labels={'Score': 'Pontuação', 'Count': 'Contagem de Avaliações'}, barmode='group', orientation='v')

### Comentários com curtidas.
* Esse gráfico está nos mostrando quais são nossos comentários com mais curtidas e em qual avaliação ele se encontra. Abaixo vamos entrar no nível do detalhe.

In [ ]:
fig = px.scatter(df, x='Score', y='Thumbs Up Count', title='Gráfico de Dispersão de Avaliações e Contagem de Curtidas',
                labels={'Score': 'Pontuação', 'Thumbs Up Count': 'Contagem de Curtidas'})

fig.update_traces(marker=dict(size=10, opacity=0.5))
fig.update_layout(hovermode='closest', hoverlabel=dict(namelength=-1))
fig.update_xaxes(title='Pontuação')
fig.update_yaxes(title='Contagem de Curtidas')

fig.show()


* Quais são nossos comentários mais curtidos? A partir dos dados que temos.

In [ ]:
pd.set_option('display.max_colwidth', None)
top_reviews = df.sort_values(by='Thumbs Up Count', ascending=False).head(10)
top_reviews.reset_index(drop=True).drop(columns=['Review ID'])

* E que tal uma núvem de palavras a partir das avaliações?

In [ ]:
text = ' '.join(df['Content'].tolist())

wordcloud = WordCloud(width=800, height=400, background_color='white',
                    max_words=100, max_font_size=100, colormap='winter').generate(text)

fig = go.Figure(go.Image(z=wordcloud.to_image(), hoverinfo='skip'))
fig.update_layout(title='Nuvem de palavras a partir dos comentários das avaliações.', showlegend=False, xaxis_visible=False, yaxis_visible=False)

fig.show()